# 🩺 AI-Powered Clinical Dashboard (Integration & Visualization Module)

This notebook implements the **Integration and Visualization Module** of the *AI-Powered Clinical Dashboard System*.

It brings together patient details, ICD-10 predictions, and AI-generated notes into one unified Streamlit dashboard.

**Objectives:**
- Centralize patient data for better accessibility.  
- Integrate AI-based insights with clinical visualization.  
- Provide an interactive web-based view s.


## ⚙️ Setup and Configuration

We begin by installing and importing all required libraries.

**Libraries used:**
- `pandas` → Handle structured healthcare data  
- `streamlit` → Build the interactive dashboard  
- `plotly.express` → Generate visual insights  



The Streamlit configuration includes a soft theme with pastel tones for readability and a clinical aesthetic.


In [1]:
!pip install streamlit pandas matplotlib plotly pyngrok --quiet
import pandas as pd
import os


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 123.1 MB/s eta 0:00:00


## 💾 Dataset Loading

The unified dataset contains all essential healthcare details:
- Patient demographic and admission data  
- Doctor, hospital, and medication information  
- AI-generated clinical notes  
- Predicted ICD-10 codes  

We use `@st.cache_data` to optimize dashboard performance and prevent reloading overhead.


In [2]:
data_path = "/content/UnifiedDataset_with_images.csv"
df = pd.read_csv(data_path)

print("✅ Data loaded successfully with", len(df), "records")
df.head(3)


✅ Data loaded successfully with 502 records


,patient_id,Name,Age,Gender,Blood Type,Medical Condition_x,Date of Admission,Discharge Date,Doctor,Hospital,Admission Type,Medication,Test Results,condition_keywords,xray_caption,prompt,clinical_note,Medical Condition_y,Predicted_ICD,xray_image
0,P1000,Bobby Jackson,30-year-old,Male,B,Cancer,2024-01-31,2024-02-02,Matthew Smith,Sons And Miller,Urgent admission,Paracetamol,Normal,cancer,the cardiomediastinal silhouette and pulmonary...,Clinical note of the patient:\nName: Bobby Jac...,Clinical note of the patient: Name: Bobby Jack...,Cancer,S05.31,/content/enhanced_images/xray_enhanced/xray_en...
1,P1001,Leslie Terry,62-year-old,Male,A+,Obesity,2019-08-20,2019-08-26,Samantha Davies,Kim Inc,Emergency admission,Ibuprofen,Inconclusive,obesity,the heart size and cardiomediastinal silhouett...,Clinical note of the patient:\nName: Leslie Te...,Clinical note of the patient: Name: Leslie Ter...,Obesity,S05.31,/content/enhanced_images/xray_enhanced/xray_en...
2,P1002,Danny Smith,76-year-old,Female,A,Obesity,2022-09-22,2022-10-07,Tiffany Mitchell,Cook Plc,Emergency admission,Aspirin,Normal,obesity,the cardiomediastinal silhouette and pulmonary...,Clinical note of the patient:\nName: Danny Smi...,Clinical note of the patient: Name: Danny Smit...,Obesity,S05.31,/content/enhanced_images/xray_enhanced/xray_en...


# 🩺 Streamlit App Layout

This section builds the **Streamlit-based dashboard** for visualizing unified healthcare data.

---

### 🎨 Design Highlights
- Clean, soft UI with cream background and blue headings.  
- Readable layout for doctors and analysts.  
- Responsive and visually minimal interface.

---

### 🧭 App Modes
1. **📋 Overview Mode** – Displays all patient records in a structured table.  
2. **🔍 Detailed View** – Shows complete details for a selected patient.

---

### 📊 Insights and Charts
- 🥧 Medical Condition Distribution (Pie Chart)  
- 📊 Top 10 ICD-10 Codes (Bar Chart)  
- 📈 Patient Admissions Over Time (Line Chart)

---

### 💡 Features
- Search and filter patients easily.  
- AI-generated notes and ICD-10 codes displayed clearly.  
- Works smoothly in **Colab + Ngrok** or local Streamlit setup.

---



## 🧩 Streamlit Application Script (app.py)

Below is the complete Streamlit app code that builds and visualizes the AI-Powered Clinical Dashboard interface.


In [34]:
%%writefile app.py
import streamlit as st
import pandas as pd
from PIL import Image
import plotly.express as px
import os

# ==============================
# 🎨 Streamlit Configuration
# ==============================
st.set_page_config(page_title="AI-Powered Clinical Dashboard", page_icon="🩺", layout="wide")

# ==============================
# 💅 Custom CSS Styling
# ==============================
st.markdown("""
<style>
body {
    background-color: #fffaf5; /* soft cream tone */
    color: #1c1e21;
    font-family: 'Segoe UI', sans-serif;
}
h1 {
    color: #004d80 !important;
    text-align: center;
    font-weight: 700;
}
h2, h3 {
    color: #004d80 !important;
}
.subtext {
    text-align: center;
    font-style: italic;
    color: #6c757d;
    margin-top: -10px;
}
.block-container {
    padding-top: 1rem;
}
.footer {
    text-align: center;
    color: #5f6368;
    background-color: #f1f3f4;
    padding: 15px;
    border-radius: 8px;
    margin-top: 30px;
}
.condition-keywords {
    background-color: #fff1b8;  /* warm yellow highlight */
    padding: 10px 14px;
    border-left: 6px solid #f4b400; /* golden edge */
    border-radius: 8px;
    color: #102a43;
    font-size: 1.2em;
    font-weight: 600;
    display: inline-block;
    margin-top: 5px;
}
.note-box {
    background-color: #eaf4ff; /* subtle blue box */
    padding: 12px;
    border-radius: 10px;
    color: #1c1e21;
    font-size: 0.95em;
    line-height: 1.5em;
}
</style>
""", unsafe_allow_html=True)

# ==============================
# 📥 Load Unified Dataset
# ==============================
@st.cache_data
def load_data():
    return pd.read_csv("/content/UnifiedDataset_with_images.csv")

df = load_data()

# ==============================
# 🏥 HEADER
# ==============================
st.title("AI-Powered Clinical Dashboard")
st.markdown("<p class='subtext'>Integrated Clinical Insights & EHR Visualization System </p>", unsafe_allow_html=True)
st.divider()

# ==============================
# 🧭 MODE TOGGLE
# ==============================
view_mode = st.radio("Choose view mode:", ["📋 Overview Table", "🔍 Detailed View"], horizontal=True)

# ==============================
# 📋 OVERVIEW MODE
# ==============================
if view_mode == "📋 Overview Table":
    st.subheader("🧾 Patient Records Overview")

    # Handle merged-column variations automatically
    condition_col = "Medical Condition"
    if "Medical Condition_x" in df.columns:
        condition_col = "Medical Condition_x"
    elif "Medical Condition_y" in df.columns:
        condition_col = "Medical Condition_y"

    display_cols = ["patient_id", "Name", "Age", "Gender", condition_col, "Predicted_ICD"]

    st.dataframe(df[display_cols].rename(columns={condition_col: "Medical Condition"}), use_container_width=True)
    st.markdown("Switch to *'Detailed View'* for in-depth visualization of each patient record.")

# ==============================
# 🔍 DETAILED VIEW MODE
# ==============================
else:
    st.sidebar.header("🔍 Search / Filter Patient")
    query = st.sidebar.text_input("Enter patient name or ID")

    filtered_df = df.copy()
    if query:
        filtered_df = df[df["Name"].str.contains(query, case=False, na=False) |
                         df["patient_id"].astype(str).str.contains(query, case=False, na=False)]

    patient = st.sidebar.selectbox("Select Patient", filtered_df["Name"].unique())
    record = filtered_df[filtered_df["Name"] == patient].iloc[0]

    # ✅ Handle column variation
    condition_col = "Medical Condition"
    if "Medical Condition_x" in df.columns:
        condition_col = "Medical Condition_x"
    elif "Medical Condition_y" in df.columns:
        condition_col = "Medical Condition_y"

    col1, col2 = st.columns([1, 1])

    # --- LEFT COLUMN (Patient Info & X-ray) ---
    with col1:
        st.subheader("🧍 Patient & Admission Details")
        st.write(f"**Patient ID:** {record['patient_id']}")
        st.write(f"**Name:** {record['Name']}")
        st.write(f"**Age / Gender:** {record['Age']} / {record['Gender']}")
        st.write(f"**Blood Type:** {record['Blood Type']}")
        st.write(f"**Condition:** {record[condition_col]}")
        st.write(f"**Admission Type:** {record['Admission Type']}")
        st.write(f"**Doctor:** {record['Doctor']}")
        st.write(f"**Hospital:** {record['Hospital']}")
        st.write(f"**Medication:** {record['Medication']}")
        st.write(f"**Date:** {record['Date of Admission']} → {record['Discharge Date']}")
        st.write(f"**Test Results:** {record['Test Results']}")



    # --- RIGHT COLUMN (Notes & ICD Info) ---
    with col2:
        st.subheader("📝 AI-Generated Clinical Note")
        st.markdown(f"<div class='note-box'>{record['clinical_note']}</div>", unsafe_allow_html=True)

        st.subheader("💊 Predicted ICD-10 Code")
        st.success(record["Predicted_ICD"])

        if "condition_keywords" in record and pd.notna(record["condition_keywords"]):
            st.subheader("🔤 Condition Keywords")
            st.markdown(f"<div class='condition-keywords'>{record['condition_keywords'].capitalize()}</div>", unsafe_allow_html=True)

    # --- CHARTS AREA ---
    st.divider()
    st.subheader("📊 Data Insights Overview")

    colA, colB, colC = st.columns(3)

    with colA:
        condition_counts = df[condition_col].value_counts().reset_index()
        condition_counts.columns = ["Condition", "Count"]
        fig1 = px.pie(condition_counts, names="Condition", values="Count",
                      title="Medical Condition Distribution",
                      color_discrete_sequence=px.colors.qualitative.Pastel)
        st.plotly_chart(fig1, use_container_width=True)

    with colB:
        icd_counts = df["Predicted_ICD"].value_counts().reset_index()
        icd_counts.columns = ["ICD Code", "Count"]
        fig2 = px.bar(icd_counts.head(10), x="ICD Code", y="Count",
                      title="Top 10 ICD-10 Codes",
                      color="Count", color_continuous_scale="Blues")
        st.plotly_chart(fig2, use_container_width=True)

    with colC:
        if "Date of Admission" in df.columns:
            timeline = df["Date of Admission"].value_counts().sort_index().reset_index()
            timeline.columns = ["Date", "Admissions"]
            fig3 = px.line(timeline, x="Date", y="Admissions", title="Patient Admissions Over Time",
                           markers=True, line_shape="spline", color_discrete_sequence=["#72aee6"])
            st.plotly_chart(fig3, use_container_width=True)

# ==============================
# 👣 FOOTER
# ==============================
st.divider()
st.markdown("""
<div class="footer">
    <h4>👩‍💻 Developed by <b>Lalithanjali Kodavali</b></h4>
    <a href="https://github.com/lalithanjali04" target="_blank">🌐 GitHub</a> |
    <a href="https://www.linkedin.com/in/lalithanjali-kodavali/" target="_blank">💼 LinkedIn</a>
</div>
""", unsafe_allow_html=True)


Overwriting app.py


## 🌐 Streamlit + Ngrok Setup

This step securely launches the Streamlit dashboard from Google Colab using **Ngrok**.

- Retrieves Ngrok authentication token from **Colab Secrets**.  
- Closes any existing sessions to avoid conflicts.  
- Creates a new tunnel to host the app on port **6006**.  
- Generates a public access URL for the live dashboard.

Once the setup completes, click the link shown in the output to open your **AI-Powered Clinical Dashboard**.


In [35]:
# ==============================
# 🌐 Secure Streamlit + Ngrok Setup (Colab Secrets Compatible)
# ==============================
from pyngrok import ngrok
from google.colab import userdata
import time
import os

# 1️⃣ Retrieve ngrok token securely from Colab Secrets
ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')

if not ngrok_authtoken:
    raise ValueError("❌ NGROK_AUTHTOKEN not found in Colab Secrets. Add it via View → Secrets → New Secret → 'NGROK_AUTHTOKEN'.")

# 2️⃣ Authenticate ngrok
try:
    ngrok.set_auth_token(ngrok_authtoken)
    print("✅ Ngrok authentication successful!")
except Exception as e:
    raise ValueError(f"❌ Failed to set ngrok authtoken. Error: {e}")

# 3️⃣ Kill any previous ngrok/Streamlit sessions to avoid conflicts
!kill -9 $(lsof -t -i:6006) 2>/dev/null || true
ngrok.kill()

# 4️⃣ Create a new tunnel to port 6006
public_url = ngrok.connect(addr=6006)
print("🌐 Streamlit App URL:", public_url)

# 5️⃣ Launch Streamlit app
!streamlit run app.py --server.port 6006 > /dev/null 2>&1 &

# Wait a few seconds for Streamlit to start
time.sleep(5)
print("\n✅ Streamlit app is starting... click the link above to open!")

✅ Ngrok authentication successful!
🌐 Streamlit App URL: NgrokTunnel: "https://unquaking-alberta-caenogenetic.ngrok-free.dev" -> "http://localhost:6006"



✅ Streamlit app is starting... click the link above to open!



---


## 🧩 Summary and Outcome

✅ Integrated patient-level and analytical insights in one dashboard.  
✅ Improved interpretability of AI predictions (ICD-10, notes).  
✅ Clean, professional, and deployable design.  
✅ Streamlit app supports interactive exploration and visual analytics.

This dashboard serves as a clinical decision support visualization tool.


## 🏁 Project Completion Note

This concludes the Integration and Visualization stage of the project. The system now offers a complete, AI-driven clinical reporting and analysis dashboard.
